<a href="https://colab.research.google.com/github/vmoctavio/TFG-vmoctavio/blob/master/TFGVOP_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Número de veces que se entrena la red
EPOCHS = 5
# Initial Learning Rate
INIT_LR = 0.00001
# Tamaño del lote
BATCH_SIZE = 32
# Tamaño de la imagen: ancho
width=256
# Tamaño de la imagen: alto
height=256
# Número de canales de la imagen
depth=3
# Porcentaje división dataset en train y test
TEST_SIZE = 0.1
# Porcentaje división dataset en train y validation (a partir del subconjunto anterior)
VALID_SIZE = 0.2
# Division del dataset no aleatorio
RANDOM_STATE = 42
# Directorio de trabajo donde está el dataset
directory_root = '/content/drive/My Drive/Dataset_vid_prueba_dest/'
# Directorio de trabajo donde se generan los logs de salida del proceso
directory_log = '/content/drive/My Drive/logs/'
# Directorio de trabajo donde se generan los modelos de salida del proceso
directory_modelos = '/content/drive/My Drive/modelos/'
# Número de registros a cargar por directorio
CONTREG = 50
# Training progress; el 1 muestra una barra de progreso animada por epoch
VERBOSE = 1

In [0]:
# -----------------------------------------------------------------------------------------------
# Imports necesarios
# -----------------------------------------------------------------------------------------------
#
# Arquitectura de red a probar
from tensorflow.keras.applications import vgg16
#
# API de alto nivel para procesos de Deep Learning
import keras
#
# Uso de funciones del backend
from keras import backend as K
#
# Uso de funciones de manejo de imágenes
from keras.preprocessing import image
#
# Convertir una imagen en una matriz NumPy 
from keras.preprocessing.image import img_to_array
#
# Convierte un vector de clase (enteros) en una matriz de clase binaria
from keras.utils import to_categorical
#
# Optimizar para compilar un modelo
from keras.optimizers import Adam
#
# Usar comandos del sistema operativo
import os
#
# Manejo de arrays 
import numpy as np
#
# Para manipular fechas y horas
import datetime
#
# Funciones matemáticas 
import math
#
# Librería para generar gráficas
import matplotlib.pyplot as plt
#
# Entorno de trabajo para redes deep learning
import tensorflow as tf
#
# Convierte un modelo de Keras a diagrama y lo guarda en un archivo
from tensorflow.keras.utils import plot_model 
#
# Dividir un dataset en dos
from sklearn.model_selection import train_test_split
#
# Visualizar la matriz de confusión - rendimiento de un algoritmo
from sklearn.metrics import confusion_matrix  
#
# Visualizar informe con las principales métricas de clasificación
from sklearn.metrics import classification_report  
#
# Librería para el análisis de datos
import pandas as pd  
#
# Librería para visualización de datos y gráficos
import seaborn as sn  
#
# Módulo para control de excepciones
import sys
#
# Devuelve una lista que contiene el nombre de las entradas en el directorio de la ruta del parámetro.
from os import listdir
#
# Librería para el procesamiento de imágenes
import cv2
#
# Librería para serializar objetos
import pickle
#


# -----------------------------------------------------------------------------------------------
# Inicio del proceso
# -----------------------------------------------------------------------------------------------
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Inicio del proceso",
    "vgg16")
#    os.path.basename(__file__))
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Inicialización de variables
# -----------------------------------------------------------------------------------------------
#
image_list=[]       # array de array's resultado de convertir las imágenes del directorio
image_labels=[]     # array de array's de etiquetas
labels=[]           # array con las distintas etiquetas del dataset
# 
# Intervalo para las coordenadas de la gráfica
INTERVALO = math.ceil(EPOCHS/10)

In [0]:
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Llamada al proceso TFGVOP_load_dataset ")

In [0]:
# Tamaño por defecto de las imágenes
default_image_size = tuple((height, width))
#
# -----------------------------------------------------------------------------------------------
# Función para redimensionar una imagen y convertirla en array
# -----------------------------------------------------------------------------------------------
#
def convert_image_to_array(image_dir):
    try:
        # Carga la imagen del fichero del parámetro
        image = cv2.imread(image_dir)
        if image is not None:   # Detecta si el fichero existe
            # Redimensiona la imagen al tamaño especificado en la variable default_image_size
            image = cv2.resize(image, default_image_size)
            return img_to_array(image) # Devuelve como array la imagen del parámetro
        else :
            return np.array([]) # Devuelve un array vacío
    except Exception as e: # Controla cualquier error que se produzca en la función
        print("[ERROR]", 
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            sys.exc_info()[0],
            sys.exc_info()[1],
            sys.exc_info()[2])
        return None
#
# -----------------------------------------------------------------------------------------------
# Proceso principal: lee las carpetas del directorio y genera los correspondientes arrays para
#                    imágenes y etiquetas
# -----------------------------------------------------------------------------------------------
#
def load_dataset_process(directory_root):
#   
    cont_labels = 0         # Contador de etiquetas diferentes
    image_label = []        # Array con el nombre de las etiquetas
    labels = []             # Array con las distintas etiquetas del dataset
    label_list = []         # Array de Array's de etiquetas
    image_list = []         # Array de Array's de imágenes
#
    try:
        root_dir = listdir(directory_root)      # Genera una lista con los directorios existentes
#
        for directory in root_dir :
            # elimina .DS_Store de la lista en caso de existir
            if directory == ".DS_Store" :
                root_dir.remove(directory)
#
        plant_disease_folder_list = listdir(f"{directory_root}") # Genera una lista con los directorios existentes
        for disease_folder in plant_disease_folder_list:
            # elimina .DS_Store de la lista en caso de existir
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)
#
        for plant_disease_folder in plant_disease_folder_list:  # Nos recorremos todos los directorios existentes
            print(f"[INFO]", 
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "Procesando el directorio... ",
                plant_disease_folder)
#
            total_images_origen = 0 # contador imágenes por cada directorio
#
            labels.append(plant_disease_folder)   # Lista con las diferentes etiquetas
            cont_labels = cont_labels + 1
#        
            # Genera una lista con todos los archivos de un directorio concreto
            plant_disease_image_list = listdir(f"{directory_root}/{plant_disease_folder}/")
# 
            for single_plant_disease_image in plant_disease_image_list:  # Nos recorremos todos los archivos de un directorio concreto
                # elimina .DS_Store de la lista en caso de existir
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)
#
            for image in plant_disease_image_list[:CONTREG]: # Nos recorremos todos los archivos de un directorio concreto
                image_directory = f"{directory_root}/{plant_disease_folder}/{image}"
                # Si el archivo tiene extensión jpg o JPG lo tratamos 
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    #  print("[INFO]",
                    #      datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    #      "Cargando imagen ... ",image)
                    #      Carga lista de imágenes y etiquetas
                    image_list.append(convert_image_to_array(image_directory))  # Convierte imagen a array
                    image_label.append(cont_labels-1)


#                    image_label.append(plant_disease_folder)
                    total_images_origen = total_images_origen + 1 #  contador imágenes por cada directorio
#
            print("[INFO]", 
                datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "Total imágenes del directorio ... ",
                plant_disease_folder,total_images_origen)
#
        print("[INFO]",
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Proceso de carga de imágenes completado ")

        #
        # -----------------------------------------------------------------------------------------------
        # Obtener el tamño de la imagen
        # -----------------------------------------------------------------------------------------------
        #
        #image_size = len(image_list) ver si esto se puede borar   ****************************
        #
        # -----------------------------------------------------------------------------------------------
        # Transforma las etiquetas de las imágenes mediante LabelBinarizer
        # -----------------------------------------------------------------------------------------------
        #
#        label_binarizer = LabelBinarizer()
#        label_list = label_binarizer.fit_transform(image_label)
        # esto creo que podría sobrar, no sé para qué es.
        # si se quita, también sobraría el input pickle
#        pickle.dump(label_binarizer,open('label_transform.pkl', 'wb'))    
    #
        print("[INFO]",
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Total Etiquetas... ",
            cont_labels)
    #       se puede borar todo lo relacionado con binarizer 
#        print("[INFO]",
#            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
#            "Etiquetas ... ",
#            label_binarizer.classes_)
        #
        # -----------------------------------------------------------------------------------------------
        # ver si esto se puede borrar
        # -----------------------------------------------------------------------------------------------
        #
        #    np_image_list = np.array(image_list, dtype=np.float16) / 225.0   # no sé para qué es lo del / 225.0
        #   hace un scalar
        # -----------------------------------------------------------------------------------------------
        # xxxxxx
        # -----------------------------------------------------------------------------------------------

    except Exception as e: # Controla cualquier error que se produzca en la función
        print("[ERROR]",
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            sys.exc_info()[0],
            sys.exc_info()[1],
            sys.exc_info()[2])
    #
    print("[INFO]", 
        datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Fin del proceso",
        "load_dataset")
    #    os.path.basename(__file__))
    #
    # -----------------------------------------------------------------------------------------------
    # Fin de la ejecución y devolución del resultado en los parámetros de salida
    # -----------------------------------------------------------------------------------------------
    #
    return(image_label,image_list,labels)
#
# -----------------------------------------------------------------------------------------------
# Fin del proceso
# -----------------------------------------------------------------------------------------------
#

In [0]:
(image_labels,image_list,labels) = load_dataset_process(directory_root)
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Imágenes cargadas. Número total de imágenes para procesar:  ",
    len(image_list),
    ". Número total de clases diferentes:",
    len(labels))

In [0]:
#
# -----------------------------------------------------------------------------------------------
# Calculamos el número de clases diferentes
# -----------------------------------------------------------------------------------------------
#
n_classes = len(labels)
#
# -----------------------------------------------------------------------------------------------
# Convertir la lista de imágenes y etiquetas en arrays Numpy
# -----------------------------------------------------------------------------------------------
#
np_image_list = np.array(image_list, dtype=np.uint8)
y = np.array(image_labels)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# División de los datos (imágenes y etiquetas) 
# en archivos para entrenamiento (train) y pruebas (test)
# -----------------------------------------------------------------------------------------------
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Dividiendo datos en train y test...")
#
x_train, x_test, y_train, y_test = train_test_split(np_image_list,          # array de imágenes
                                                    y,                      # array de etiquetas
                                                    test_size=TEST_SIZE,    # % para el archivo de test 
                                                    random_state = RANDOM_STATE)
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Training data shape (x_train): ",
    x_train.shape)
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Training data shape (y_train): ",
    y_train.shape)
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Testing data shape (x_test): ",
    x_test.shape)
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Testing data shape (y_test): ",
    y_test.shape)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Cambiamos los tipos y escalamos los pixeles en el ranto [0,1]
# -----------------------------------------------------------------------------------------------
#
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.
x_test = x_test / 255.
#
# -----------------------------------------------------------------------------------------------
# Convierte un vector de clase (enteros) en una matriz de clase binaria
# -----------------------------------------------------------------------------------------------
# 
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# División de los datos (imágenes y etiquetas) 
# en archivos para entrenamiento (train) y validación (valid)
# a partir de los archivos de entrenamiento que se generaron en la primera división.
# -----------------------------------------------------------------------------------------------
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Dividiendo datos en train y valid...")
#
x_train, x_valid, train_label, valid_label = train_test_split(x_train,          # array de imágenes 
                                                    y_train_one_hot,            # array de etiquetas
                                                    test_size=VALID_SIZE,       # % para el archivo de validación  
                                                    random_state = RANDOM_STATE)
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Training data shape (x_train): ",
    x_train.shape)
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Training data shape (train_label): ",
    train_label.shape)
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Validating data shape (x_valid): ",
    x_valid.shape)
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Validating data shape (valid_label): ",
    valid_label.shape)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Inicialización archivo de registro (LOG_DIR) y Tensorboard 
# -----------------------------------------------------------------------------------------------
#
LOG_DIR=directory_log + 'vgg16_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Frecuencia (en epochs) a la que se calculan los histogramas de activación para las capas del modelo.
v_histogram_freq=1  
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Generado log en... ",
    LOG_DIR)
#
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR,      # Directorio de registro
                                                    histogram_freq=v_histogram_freq,       
                                                    write_graph=True,       # Si visualizar el gráfico
                                                    write_images=True,      # Si visualizar imágenes
                                                    update_freq='epoch',    # Las métricas se generan por cada epoch
                                                    profile_batch=2)        # Perfilar el segundo lote
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Inicializa input_shape en función del formato de la imagen
# -----------------------------------------------------------------------------------------------
#
inputShape = (height, width, depth)
chanDim = -1
if K.image_data_format() == "channels_first":
    inputShape = (depth, height, width)
    chanDim = 1
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Función para crear el modelo vgg16
# -----------------------------------------------------------------------------------------------
#
def create_vgg16():
    model = vgg16.VGG16(include_top=True,       # Incluir las 3 capas totalmente conectadas en la parte superior de la red
                        weights=None,           # Inicialización random (no partir de pre-entrenadas de imagenet)
                        input_tensor=None,      # No usar tensor como entrada de imágenes
                        input_shape=inputShape, # Resolución de las imágenes de entrada
                        pooling=None,           # Modo agrupación de características
                        classes=n_classes)      # Número de clases
    return model
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Llamada a la función para crear el modelo
# -----------------------------------------------------------------------------------------------
#
vgg16_model = create_vgg16()  
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Inicializa parámetros para el optimizador ADAM 
# -----------------------------------------------------------------------------------------------
#
opt = tf.keras.optimizers.Adam(lr=INIT_LR,                  # Initial Learning Rate
                               decay=INIT_LR / EPOCHS)      # Disminución de Learning Rate 
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Configuración / compilación del proceso de aprendizaje
# -----------------------------------------------------------------------------------------------
#
vgg16_model.compile(loss='categorical_crossentropy',        # Función de pérdida
                    optimizer=opt,                          # Optimizador 
                    metrics=['acc', 'mean_squared_error'])  # Métricas del proceso 
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Imprime una representación resumida del modelo
# -----------------------------------------------------------------------------------------------
#
vgg16_model.summary()
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Creamos el directorio destino de las gráficas, si no existe 
# -----------------------------------------------------------------------------------------------
try:
    os.stat(directory_log)
except:
    os.mkdir(directory_log)
#
try:
    os.stat(directory_log + 'modelos')
except:
    os.mkdir(directory_log + 'modelos')
#
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Convierte un modelo de Keras a diagrama y lo guarda en un archivo
# -----------------------------------------------------------------------------------------------
#
tf.keras.utils.plot_model(model=vgg16_model,
                       to_file=directory_log + 'modelos/' + 'Plotmodel_vgg16.png',
                       show_shapes=True,
                       show_layer_names=True,
                       rankdir='TB',
                       expand_nested=True,
                       dpi=96)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Ejecución / entrenamiento de la red
# -----------------------------------------------------------------------------------------------
#
vgg16 = vgg16_model.fit(x_train,                                    # Imágenes de entrenamiento
                        train_label,                                # Etiquetas entrenamiento 
                        batch_size=BATCH_SIZE,                      # Tamaño del lote 
                        epochs=EPOCHS,                              # Número de veces que se entrena la red
                        verbose=VERBOSE,                            # Barra de progreso 
                        validation_data=(x_valid, valid_label),     # Datos de validación (imágenes y etiquetas)
                        shuffle=True,                               # Reordenar los lotes al comienzo de cada epoch
                        callbacks=[tensorboard_callback])           # Configuración de Tensorboard   
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Representación de las métricas de entrenamiento y validación
# -----------------------------------------------------------------------------------------------
#                               Training Accuracy vs Validation Accuracy
plt.figure(0)  
plt.plot(vgg16.history['acc'],'r')  
plt.plot(vgg16.history['val_acc'],'g')  
plt.xticks(np.arange(0, EPOCHS, INTERVALO))
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Accuracy")  
plt.title("Training Accuracy vs Validation Accuracy en vgg16")  
plt.legend(['train','validation'])
plt.savefig(directory_log + 'modelos/' + 'TrainingAccuracyvsValidationAccuracy_en_vgg16.png')  

#                               Training Loss vs Validation Loss
plt.figure(1)  
plt.plot(vgg16.history['loss'],'r')  
plt.plot(vgg16.history['val_loss'],'g')  
plt.xticks(np.arange(0, EPOCHS, INTERVALO))
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("Loss")  
plt.title("Training Loss vs Validation Loss en vgg16")  
plt.legend(['train','validation'])
plt.savefig(directory_log + 'modelos/' + 'TrainingLossvsValidationLoss_en_vgg16.png')  

#                               Training mse vs Validation mse
plt.figure(2)  
plt.plot(vgg16.history['mean_squared_error'],'r')  
plt.plot(vgg16.history['val_mean_squared_error'],'g')  
plt.xticks(np.arange(0, EPOCHS, INTERVALO))
plt.rcParams['figure.figsize'] = (8, 6)  
plt.xlabel("Num of Epochs")  
plt.ylabel("mean_squared_error")  
plt.title("Training mse vs Validation mse en vgg16")  
plt.legend(['train','validation'])
plt.savefig(directory_log + 'modelos/' + 'TrainingmsevsValidationmse_en_vgg16.png')  
#
plt.show()
#
# ver por qué da error el mse.....

In [0]:
# -----------------------------------------------------------------------------------------------
# Calcular la precisión del modelo con el conjunto de datos de test 
# los cuales no han participado en el proceso de entrenamiento y validación
# -----------------------------------------------------------------------------------------------
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Calculando la precisión del modelo vgg16... ")
#
resultado_test = vgg16_model.evaluate(x_test,
                              y_test_one_hot)
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    f"Test Accuracy vgg16: {resultado_test[1]*100}")
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    f"Test Loss vgg16: {resultado_test[0]*100}")  # no sé si el loss se multiplica o no
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Genera las predicciones de salida para el conjunto de datos test
# -----------------------------------------------------------------------------------------------
#
predicted_classes_test = vgg16_model.predict(x_test)
#
predicted_classes=[]
for predicted_image in predicted_classes_test:
    predicted_classes.append(predicted_image.tolist().index(max(predicted_image)))
#
predicted_classes=np.array(predicted_classes)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Convertir array de etiquetas de test
# -----------------------------------------------------------------------------------------------
#
y_test_aux = np.array(y_test)
#
predicted_classes.shape, y_test.shape
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Calcular predicciones correctas
# -----------------------------------------------------------------------------------------------
#
correct = np.where(predicted_classes==y_test)[0]
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Se han encontrado %d etiquetas correctas" % len(correct))
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Ejemplos de predicciones correctas
# -----------------------------------------------------------------------------------------------
#
for i, correct in enumerate(correct[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[correct].reshape(256,256,3), cmap='gray', interpolation='none')
    plt.title("{} / {}".format(labels[predicted_classes[correct]],labels[y_test[correct]]))
    plt.tight_layout()
#
plt.savefig(directory_log + 'modelos/' + 'Ejemploprediccionescorrectas_en_vgg16.png')  
plt.show()  
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Calcular predicciones incorrectas
# -----------------------------------------------------------------------------------------------
#
incorrect = np.where(predicted_classes!=y_test)[0]
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Se han encontrado %d etiquetas incorrectas" % len(incorrect))
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Ejemplos de predicciones incorrecatas
# -----------------------------------------------------------------------------------------------
#
# print("Calculado / Correcto")
for i, incorrect in enumerate(incorrect[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(x_test[incorrect].reshape(256,256,3), cmap='gray', interpolation='none')
    plt.title("{} / {}".format(labels[predicted_classes[incorrect]],labels[y_test[incorrect]]))
    plt.tight_layout()
#
plt.savefig(directory_log + 'modelos/' + 'Ejemploprediccionesincorrectas_en_vgg16.png')  
plt.show()  
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Matriz de confusión para evaluación de falsos positivos y falsos negativos
# -----------------------------------------------------------------------------------------------
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Creando la matriz de confusión vgg16...")
#
# -----------------------------------------------------------------------------------------------
# Creamos la matriz de confusión
# -----------------------------------------------------------------------------------------------
#
predicted_classes_test_confusion = np.argmax(predicted_classes_test, axis=1)  
#
predicted_classes_test_confusion_cm = confusion_matrix(np.argmax(y_test_one_hot, axis=1),
                                                       predicted_classes_test_confusion)
#
print("[INFO]",
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Visualizando la matriz de confusión...")
#
# -----------------------------------------------------------------------------------------------
# Visualizamos la matriz de confusión. Aquí se visualiza el desempeño del algoritmo.
# Cada columna de la matriz representa el número de predicciones de cada etiqueta y 
# las filas la instancia de la etiqueta real.
# -----------------------------------------------------------------------------------------------
#
predicted_classes_test_confusion_cm_df = pd.DataFrame(predicted_classes_test_confusion_cm,
                                                      columns=labels,
                                                      index=labels)
#
plt.figure(figsize = (20,14))  
#
sn.set(font_scale=2)                # Tamaño de fuente mapa calor
sn.heatmap(predicted_classes_test_confusion_cm_df,
           annot=True,              # Escribe el número de coincidencias en cada celda
           linewidth=0.5,           # Ancho del borde de celdas de la tabla
           cmap="YlOrRd",           # Mapa de color amarillo - naranja - rojo
           square=True,             # Forzar tamaño celdas
           annot_kws={"size": 20})  # Tamaño de fuente barra
plt.title('Matriz de confusión vgg16', pad=100, fontsize = 30, color='Black', fontstyle='italic')
plt.savefig(directory_log + 'modelos/' + 'Matrizdeconfusion_en_vgg16.png')  

plt.show() 
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Visualizamos informe resumen de clasificación
# -----------------------------------------------------------------------------------------------
#
cm_report = classification_report(np.argmax(y_test_one_hot, axis=1),
                                predicted_classes_test_confusion,
                                target_names = labels)  
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Informe de clasificación")
print(cm_report)
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Inicio del proceso
# -----------------------------------------------------------------------------------------------
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Inicio del proceso",
    "save_model")

#    os.path.basename(__file__))
#
# -----------------------------------------------------------------------------------------------
# Proceso principal: guarda las estadísticas del modelo "model" 
# en un archivo con nombre "model_name"
# -----------------------------------------------------------------------------------------------
#
def save_model(directory_log,model_name,model):
#
    try:
        #creamos el directorio destino si no existe 
        try:
            os.stat(directory_log + 'modelos')
        except:
            os.mkdir(directory_log + 'modelos')
#
        with open(directory_log + 'modelos/' + model_name + '.stats', 'wb') as file_modelo:  
            pickle.dump(model.history, file_modelo)
#
        print("[INFO]", 
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "Modelo correctamente guardado... ",
            model_name)
#
    except Exception as e: # Controla cualquier error que se produzca en la función
        print("[ERROR]",
            datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            sys.exc_info()[0],
            sys.exc_info()[1],
            sys.exc_info()[2])
#
    print("[INFO]", 
        datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Fin del proceso",
        "save_model")
    #    os.path.basename(__file__))
    #
    # -----------------------------------------------------------------------------------------------
    # Fin de la ejecución
    # -----------------------------------------------------------------------------------------------
    #
    return
#
# -----------------------------------------------------------------------------------------------
# Fin del proceso
# -----------------------------------------------------------------------------------------------
#

In [0]:
# -----------------------------------------------------------------------------------------------
# Guardamos el modelo para después poder compararlos
# -----------------------------------------------------------------------------------------------
#
print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Llamada al proceso TFGVOP_save_model ")
#
save_model(directory_log,'vgg16',vgg16)

print("[INFO]", 
    datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "Fin del proceso",
     "vgg16")  
#    os.path.basename(__file__))